In [ ]:
# Reproducible Analysis

## CASA0013: Foundations of Spatial Data Science

### Student Ids: ucfnlui, ucftYYYY, etc.

In [ ]:
### ucfnlui code:

In [1]:
print("start from here")

start from here


In [ ]:
### ucfniul code:

In [ ]:
print("start from here")

In [ ]:
### ucfnply code:

In [ ]:
print("start from here")

In [ ]:
### ucfnjin code:

In [ ]:
print("start from here")